In [0]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [0]:
def adv_train(model, device, train_loader, optimizer, epoch, epsilon=0.1, alpha=0.5):
    log_interval = 10
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        #print(data.shape)
        data.requires_grad = True
        
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward(retain_graph=True)

        # Generating Adversarial Examples
        # Collect gradient of inputs
        data_grad = data.grad.data

        # FGSM Attack
        perturbed_data = fgsm_attack(data, epsilon, data_grad)

        # Get loss from perturbed data
        adv_output = model(perturbed_data)
        adv_loss = F.nll_loss(adv_output, target)


        optimizer.zero_grad()

        # Calculate total loss
        total_loss = alpha*loss + (1-alpha)*adv_loss
        total_loss.backward()

        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [0]:
def train(model, device, train_loader, optimizer, epoch, epsilon=0.1, alpha=0.5):
    log_interval = 10
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() 
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [0]:
def fgsm_attack(image, epsilon, data_grad):
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon*sign_data_grad
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # Return the perturbed image
    return perturbed_image

In [0]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
def adv_test(model, device, test_loader, epsilon=0.1):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
def main():
    # Training settings
    batch_size = 100
    test_batch_size = 1000
    lr = 1.0
    gamma = 0.7 # learning rate decay
    epochs = 14
    save_model = True

    use_cuda = torch.cuda.is_available()
    print("CUDA? " + str(use_cuda))

    torch.manual_seed(20294798)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=test_batch_size, shuffle=True, **kwargs)

    model = Net().to(device)
    model_adv = Net().to(device)

    optimizer = optim.Adadelta(model.parameters(), lr=lr)
    optimizer_adv = optim.Adadelta(model_adv.parameters(), lr=lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        adv_train(model_adv, device, train_loader, optimizer_adv, epoch)
        print("Test on standard model:")
        test(model, device, test_loader)
        print("Test on adversarially trained model:")
        test(model_adv, device, test_loader)
        
        scheduler.step()

    if save_model:
        torch.save(model.state_dict(), "mnist_cnn.pt")
        torch.save(model_adv.state_dict(), "mnist_cnn_adv.pt")

In [44]:
main()

CUDA? True
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.307258
Train Epoch: 1 [1000/60000 (2%)]	Loss: 1.325261
Train Epoch: 1 [2000/60000 (3%)]	Loss: 0.690761
Train Epoch: 1 [3000/60000 (5%)]	Loss: 0.479493
Train Epoch: 1 [4000/60000 (7%)]	Loss: 0.335729
Train Epoch: 1 [5000/60000 (8%)]	Loss: 0.262178
Train Epoch: 1 [6000/60000 (10%)]	Loss: 0.321821
Train Epoch: 1 [7000/60000 (12%)]	Loss: 0.367413
Train Epoch: 1 [8000/60000 (13%)]	Loss: 0.238994
Train Epoch: 1 [9000/60000 (15%)]	Loss: 0.166172
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.173302
Train Epoch: 1 [11000/60000 (18%)]	Loss: 0.147516
Train Epoch: 1 [12000/60000 (20%)]	Loss: 0.249886
Train Epoch: 1 [13000/60000 (22%)]	Loss: 0.340570
Train Epoch: 1 [14000/60000 (23%)]	Loss: 0.135423
Train Epoch: 1 [15000/60000 (25%)]	Loss: 0.141237
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.149779
Train Epoch: 1 [17000/60000 (28%)]	Loss: 0.224169
Train Epoch: 1 [18000/60000 (30%)]	Loss: 0.102723
Train Epoch: 1 [19000/60000 (32%)]	Loss: 0.274799
Train Ep